# Collision Avoidance - Data Collection（データの収集)

basic motionのノートブックを実行し、JetBotを簡単に動かすことができ、お楽しみいただけたのではないでしょうか？ それはとてもいい経験です。でも、もっとすごいのは、JetBotは、自律的に動き回る事ができることです。

完全な自律走行はとてもハードなタスクで、この分野には多くの異なるアプローチが存在します。自律走行で直面する多くの問題は、より簡単で小さな問題に分割する事ができます。中でも最も重要で、解決すべき重要な問題は、ロボットが危険な状況に入るのを防ぐ事です。これを*collision avoidance*と呼びます。

この章では、非常に用途が広く、単体で動作するセンサーであるカメラと、ディープラーニングを用いてこの問題を解決することを経験できます。ニューラルネットワーク、カメラ、およびNVIDIA Jetson Nanoを使用して、JetBotに障害物を回避させる方法を学ぶことができます。

衝突を回避するためのアプローチは、JetBotの周りに仮想的な"safety bubble"を作り出す事でおこないます。"safety bubble"の中では、JetBotは、オブジェクトにぶつかることなく（または棚から落ちるなどのその他の危険な状況にならずに）円を描くように回転できます。

もちろん、JetBotはカメラに写るものしか認識することができないため、背後などの障害物を回避することはできません。しかし、JetBotがこれらの回避不能なシナリオに入るのを防ぐことはできます。

この方法は、実際やってみるととてもシンプルです。

最初に、"safety bubble"に違反する場所、つまり旋回したい場所にJetBotを手動で移動します。そして、``blocked``のラベルをつけます。ラベルとともに、JetBotが見ている画像もsnapshotとして保存します。

次に、JetBotを直進できる場所に手動で移動します。そして、``free``のラベルをつけます。同様に、ラベルとともにsnapshotを保存します。　

この`data_collection.ipynb`では、このようにして`free`と`blocked`のデータを集めます。ラベルと画像をたくさん用意できれば、表示される画像に基づいてJetBotの"safety bubble"が侵害されているかどうかを正確に予測できるようになります。

> 重要なメモ: Jetbotの旋回は、ロボット筐体の中心ではなく、2つのWheelの中心で回転します。これは、ロボットの"safety bubble"が侵害されているかどうかを推定する場合に、覚えておくべき重要事項です。正確である必要はないので、心配はしないでください。ぶつかりそうな場合は、より大きな"safety bubble"を想定してください。JetBotが狭い場所に入ってしまい、Uターン出来なくなるシナリオに入らないように注意します。

本サンプルを実行するにあたり、まずnvargus-daemon(カメラ等で使用)をリスタートします。

In [ ]:
!echo jetbot | sudo -S systemctl restart nvargus-daemon

カメラの初期化と表示をおこないます。

JetBotのカメラクラスは、CSI MIPI cameraを有効にするために使います。衝突回避に使うニューラルネットワークのモデルでは、224x224ピクセルの画像データを入力として使います。画像サイズが大きくなると、ニューラルネットワークモデルの学習や実行に必要とするメモリ量と処理時間が大幅に増えます。Jetson Nanoでは最適な値としてこのサイズを設定します。

カメラ画像とカメラ画像にターゲットを追加したウィジェットを作成します。

In [ ]:
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display
from jetbot import Camera, bgr8_to_jpeg

camera = Camera.instance(width=224, height=224, fps=2)

image = widgets.Image(format='jpeg', width=224, height=224)  # this width and height doesn't necessarily have to match the camera

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

次のコードを実行すると、カメラ映像を表示します。

In [ ]:
display(image)

次は、データを保存するためのディレクトリを作成しましょう。\
2つのサブフォルダ``free``と``blocked``を持つ、``dataset``フォルダを作成します。ここに、それぞれのシナリオ用の画像を置いていきます。

In [ ]:
import os

blocked_dir = 'dataset/blocked'
free_dir = 'dataset/free'

# we have this "try/except" statement because these next functions can throw an error if the directories exist already
try:
    os.makedirs(free_dir)
    os.makedirs(blocked_dir)
except FileExistsError:
    print('Directories not created becasue they already exist')

Jupyterの左側のファイルブラウザをリフレッシュすれば、これらのディレクトリが、新規で生成された事がわかります。\
次に、それぞれのクラスのラベルに対応したsnapshotを保存するためのボタンと、保存済みの各カテゴリの画像の数を表示するテキストボックスを用意します。

In [ ]:
button_layout = widgets.Layout(width='128px', height='64px')
free_button = widgets.Button(description='add free', button_style='success', layout=button_layout)
blocked_button = widgets.Button(description='add blocked', button_style='danger', layout=button_layout)
free_count = widgets.IntText(layout=button_layout, value=len(os.listdir(free_dir)))
blocked_count = widgets.IntText(layout=button_layout, value=len(os.listdir(blocked_dir)))

このボタンウィジェットは、クリックされた時に何を実行するのかまだ定義されていません。\
そこでボタンの``on_clock``イベントにそれぞれのカテゴリにイメージを保存する関数を対応づけます。\
`image = widgets.Image(format='jpeg', width=224, height=224)`で定義された`image`変数は`image.value`にJPEG形式の画像データを持っているので、それをjpgファイルとして保存します。

(異なるマシン間でも) 同じファイル名にならないうようにPythonの``uuid``パッケージを使います。このユニークな識別子は、現在時刻とマシンなどの情報から生成されます。

In [ ]:
from uuid import uuid1

def save_snapshot(directory):
    image_path = os.path.join(directory, str(uuid1()) + '.jpg')
    with open(image_path, 'wb') as f:
        f.write(image.value)

def save_free():
    global free_dir, free_count
    save_snapshot(free_dir)
    free_count.value = len(os.listdir(free_dir))
    
def save_blocked():
    global blocked_dir, blocked_count
    save_snapshot(blocked_dir)
    blocked_count.value = len(os.listdir(blocked_dir))
    
# attach the callbacks, we use a 'lambda' function to ignore the
# parameter that the on_click event would provide to our function
# because we don't need it.
free_button.on_click(lambda x: save_free())
blocked_button.on_click(lambda x: save_blocked())

# 操作ボタンを作成
画像データを集めやすくするために、basic motionと同じようにボタン操作でJetBotを操作出来るように準備します。\
ボタンは後ほどカメラ画面と一緒に表示します。

In [ ]:
import time
from jetbot import Robot
robot = Robot()

def stop(change):
    robot.stop()

# カーペットの上だと旋回出来ないことがあるので、少し出力を上げます。
def step_forward(change):
    robot.forward(0.6)
    time.sleep(0.3)
    robot.stop()

def step_backward(change):
    robot.backward(0.6)
    time.sleep(0.3)
    robot.stop()

def step_left(change):
    robot.left(0.6)
    time.sleep(0.1)
    robot.stop()

def step_right(change):
    robot.right(0.6)
    time.sleep(0.1)
    robot.stop()

# create control buttons
button_layout = widgets.Layout(width='100px', height='80px', align_self='center')
stop_button = widgets.Button(description='stop', button_style='danger', layout=button_layout)
forward_button = widgets.Button(description='forward', layout=button_layout)
backward_button = widgets.Button(description='backward', layout=button_layout)
left_button = widgets.Button(description='left', layout=button_layout)
right_button = widgets.Button(description='right', layout=button_layout)

# link buttons to actions
stop_button.on_click(stop)
forward_button.on_click(step_forward)
backward_button.on_click(step_backward)
left_button.on_click(step_left)
right_button.on_click(step_right)

## 画面と操作ボタンを表示して始めよう
次のコードを実行すると、画面とボタンが表示されます。
1. 旋回したい場所にJetBotを起き``add blocked``を押します。
2. 直進できる場所にJetBotを起き``add free``を押します。
3. 1, 2の作業を繰り返します。

ラベル付けのためにいくつかのTipsがあります。

1. さまざまな場所、角度での試行
2. 異なる照明、環境での試行
3. さまざまな壁、棚、人の足、ダンボールなどの障害物での試行
4. 異なる床の模様やパターン、なめらかさや、ガラスなどでの試行
5. データ件数はかたよりすぎないようにバランスをとる

> 最終的に、JetBotが現実の世界で遭遇するシナリオのデータが多いほど、衝突回避の挙動は正確になります。
大量のデータだけでなく、（上記のヒントで説明したような）*さまざまな*データを取得する事が重要で、各クラスの画像が少なくとも100枚以上、必要になります。

`blocked`と`free`のどちらか一方にデータがかたより過ぎると、学習時にデータ件数が多い方だけを覚えてしまいます（片方だけを予測した方が精度がよくなってしまうため）。どちらも100枚程度のデータがあればうまく学習できると思います。

In [ ]:
# control buttons
middle_box = widgets.HBox([left_button, stop_button, right_button], layout=widgets.Layout(align_self='center'))
controls_box = widgets.VBox([forward_button, middle_box, backward_button])

display(widgets.HBox([
    widgets.VBox([
        image,
        widgets.HBox([free_count, free_button]),
        widgets.HBox([blocked_count, blocked_button])
        ]),
    # display buttons
    widgets.VBox([
        controls_box
    ])
]))

## Next(次)

次は、``train_model.ipynb``で学習をおこないます。\
ノートブックメニューから`Kernel`->`Restert Kernel`を選んでJupyter kernelを再起動するか、JetBotを一度再起動してから次に進むとスムーズに進行できます。